**Notebook preparation and importing libraries**

In [1]:
from deps import load_local_libraries
load_local_libraries()

In [2]:
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CP_HUMID_AIR
from hvac.logging import ModuleLogger

In [3]:
logger = ModuleLogger.get_logger('hvac.fluids.fluid')
logger.setLevel(ModuleLogger.ERROR)

In [4]:
Q_ = Quantity

# Preliminary heat exchanger calculations

- An air stream must be cooled from 24 °C (RH 50%) to 13 °C. The mass flow rate of air is 1500 kg/hr.
- The refrigerant is R134a.
- Outdoor air temperature is 35 °C.

Define the type of refrigerant:

In [5]:
R134a = Fluid('R134a')

Define the air state at the entrance and exit of the evaporator:

In [6]:
evp_air_in = HumidAir(Tdb=Q_(24, 'degC'), RH=Q_(50, 'pct'))
evp_air_out = HumidAir(Tdb=Q_(13, 'degC'), RH=Q_(90, 'pct'))

It is assumed that air is close to saturation at the exit of the evaporator.

Set the mass flow rate of air through the evaporator:

In [7]:
evp_m_dot_air = Q_(1500, 'kg/hr')

## Heat absorption rate

The heat rate to be extracted from the air stream can now be calculated:

In [8]:
evp_Q = evp_m_dot_air * (evp_air_in.h - evp_air_out.h)
print(f"{evp_Q.to('kW'):~P.3f}")

5.657 kW


## Evaporation temperature and pressure

Based on a certain heat transfer effectiveness of the evaporator, an estimation of the refrigerant evaporating temperature can be made: 

In [9]:
evp_eps = 0.8

The theoretically maximum heat transfer in that case would be:

In [10]:
evp_Q_max = evp_Q / evp_eps
print(f"{evp_Q_max.to('kW'):~P.3f}")

7.072 kW


From the definition of maximum possible heat transfer rate in case of a wet surface, the enthalpy of saturated humid air having a temperature equal to this of the entering refrigerant can be determined:

In [11]:
evp_h_air_sat_out = evp_air_in.h - evp_Q_max / evp_m_dot_air

The complete state of this saturated air can then be determined with:

In [12]:
evp_air_sat_out = HumidAir(h=evp_h_air_sat_out, RH=Q_(100, 'pct'))

The dry-bulb temperature of this state equals the temperature of the entering refrigerant:

In [13]:
evp_T_rfg_in = evp_air_sat_out.Tdb
print(f"{evp_T_rfg_in.to('degC'):~P.2f}")

10.67 °C


Refrigerant enters the evaporator as a saturated liquid/vapor mixture. During the boiling process the temperature and pressure of the refrigerant remain constant. We define the saturated liquid state of the refrigerant as:

In [14]:
evp_rfg_sat_liq = R134a(T=evp_T_rfg_in, x=Q_(0.0, 'pct'))

The saturation pressure is also called the evaporator or evaporation pressure:

In [15]:
print(f"{evp_rfg_sat_liq.P.to('bar'):~P.1f}")

4.2 bar


## Condensation temperature and pressure

The outdoor air temperature at the entrance of the condenser is known:

In [16]:
T_outdoor = Q_(35, 'degC')

We assume a temperature difference between the outdoor air and the condenser saturation temperature: 

In [17]:
dT_cnd = Q_(20, 'K')

The condenser saturation temperature is then:

In [18]:
cnd_T_rfg_sat = T_outdoor.to('K') + dT_cnd
print(f"{cnd_T_rfg_sat.to('degC'):~P.2f}")

55.00 °C


The state of the saturated refrigerant liquid in the condenser can now be defined:

In [19]:
cnd_rfg_sat_liq = R134a(T=cnd_T_rfg_sat, x=Q_(0.0, 'frac'))

The condenser pressure is: 

In [20]:
print(f"{cnd_rfg_sat_liq.P.to('bar'):~P.1f}")

14.9 bar


## Mass flow rate of refrigerant

We want that the refrigerant leaves the condenser as a subcooled liquid (to get a good refrigeration effect) and we want that it leaves the evaporator as a superheated vapor (to avoid liquid droplets in the compressor). We select the degree of subcooling and superheat:

In [21]:
dT_sc, dT_sh = Q_(10, 'K'), Q_(10, 'K')

With these we can define the state of refrigerant leaving the condenser, entering the evaporator, and leaving the evaporator:

In [22]:
cnd_rfg_out = R134a(T=cnd_rfg_sat_liq.T - dT_sc, P=cnd_rfg_sat_liq.P)
evp_rfg_in = R134a(h=cnd_rfg_out.h, P=evp_rfg_sat_liq.P)
evp_rfg_out = R134a(T=evp_rfg_sat_liq.T + dT_sh, P=evp_rfg_sat_liq.P)

Notice that the enthalpy of refrigerant entering the evaporator is taken to be equal to the enthalpy of refrigerant leaving the condenser. This is because the expansion process between the outlet of the condenser and the inlet of the evaporator can be considered as an isenthalpic process.

Knowing the evaporator heat absorption rate and the inlet and outlet states of refrigerant at the evaporator, the mass flow rate of refrigerant can be determined:

In [23]:
m_dot_rfg = evp_Q / (evp_rfg_out.h - evp_rfg_in.h)
print(f"{m_dot_rfg.to('kg / hr'):~P.3f}")

135.570 kg/h


## State of refrigerant entering the condenser

If we assume isentropic compression of refrigerant, the state of refrigerant entering the condenser can be determined with:

In [24]:
cnd_rfg_in = R134a(P=cnd_rfg_sat_liq.P, s=evp_rfg_out.s)

The temperature of refrigerant entering the condenser in case of isentropic compression would be equal to:

In [25]:
print(f"{cnd_rfg_in.T.to('degC'):~P.2f}")

68.39 °C


In reality the compression process is not perfectly efficient. We assume the compression efficiency (i.e. the ratio of isentropic to actual work) to be: 

In [26]:
eta_cmp = 0.80

The isentropic compression work rate can be determined with:

In [27]:
W_cmp_is = m_dot_rfg * (cnd_rfg_in.h - evp_rfg_out.h)
print(f"{W_cmp_is.to('kW'):~P.3f}")

1.038 kW


The actual (irreversible adiabatic) compression work rate then follows from:

In [28]:
W_cmp_act = W_cmp_is / eta_cmp
print(f"{W_cmp_act.to('kW'):~P.3f}")

1.298 kW


The enthalpy of refrigerant at the inlet of the condenser:

In [29]:
cnd_h_rfg_in = evp_rfg_out.h + W_cmp_act / m_dot_rfg

Knowing the enthalpy and pressure, the state of refrigerant at the condenser inlet can be determined:

In [30]:
cnd_rfg_in = R134a(P=cnd_rfg_sat_liq.P, h=cnd_h_rfg_in)

The temperature of refrigerant entering the condenser in case of actual compression is equal to:

In [31]:
print(f"{cnd_rfg_in.T.to('degC'):~P.2f}")

74.32 °C


## Mass flow rate of condenser air

The heat rejection rate of the condenser can be determined:

In [32]:
cnd_Q = evp_Q + W_cmp_act
print(f"{cnd_Q.to('kW'):~P.3f}")

6.955 kW


We assume the temperature rise of air between the inlet and outlet of the condenser to be: 

In [33]:
cnd_dT_air = Q_(10, 'K')

The air inlet temperature is known and the condensation temperature was selected based on this inlet air temperature. We choose the temperature rise of air, such that the air outlet temperature is somewhere halfway between the air inlet and condensation temperature.

In [34]:
cnd_T_air_out = T_outdoor.to('K') + cnd_dT_air

Now the mass flow rate of condenser air can be determined with:

In [35]:
cnd_m_dot_air = cnd_Q / (CP_HUMID_AIR * cnd_dT_air)
print(cnd_m_dot_air.to('kg / hr'))

2454.819572123505 kilogram / hour


## Frontal area of the evaporator

We select the face velocity of air:

In [36]:
evp_v_fa = Q_(2, 'm / s')

The volume flow rate of air can be determined with:

In [37]:
evp_V_dot_air = evp_m_dot_air / evp_air_in.rho
print(f"{evp_V_dot_air.to('m ** 3 / hr'):~P.3f}")

1281.170 m³/h


The frontal area of the evaporator follows from:

In [38]:
evp_A_fr = evp_V_dot_air / evp_v_fa
print(f"{evp_A_fr.to('m ** 2'):~P.3f}")

0.178 m²


We choose an aspect ratio for the frontal area (ratio of height L3 to width L1):

In [39]:
evp_AR_A_fr = 1 / 4

The width of the frontal area can then be derived from `evp_A_fr = L1 * L3 = L1 * L1 * evp_AR_A_fr`:

In [40]:
evp_L1 = (evp_A_fr / evp_AR_A_fr) ** 0.5
print(f"{evp_L1.to('mm'):~P.0f}")

844 mm


The height of the frontal area:

In [41]:
evp_L3 = evp_AR_A_fr * evp_L1
print(f"{evp_L3.to('mm'):~P.0f}")

211 mm


## Frontal area of the condenser

In the same way, we can determine the frontal area of the condenser and its dimensions:

In [42]:
cnd_v_fa = Q_(4, 'm / s')

Assume the outdoor air has a relative humidity of 30 %:

In [43]:
cnd_air_in = HumidAir(Tdb=T_outdoor, RH=Q_(30, 'pct'))

The volume flow rate of air at the condenser inlet is then:

In [44]:
cnd_V_dot_air = cnd_m_dot_air / cnd_air_in.rho
print(f"{cnd_V_dot_air.to('m ** 3 / hr'):~P.3f}")

2178.817 m³/h


From this the frontal area of the condenser is determined as:

In [45]:
cnd_A_fr = cnd_V_dot_air / cnd_v_fa
print(f"{cnd_A_fr.to('m ** 2'):~P.3f}")

0.151 m²


We choose an aspect ratio for the frontal area (ratio of height L3 to width L1):

In [46]:
cnd_AR_A_fr = 1 / 2

The width of the frontal area can then be derived from:

In [47]:
cnd_L1 = (cnd_A_fr / cnd_AR_A_fr) ** 0.5
print(f"{cnd_L1.to('mm'):~P.0f}")

550 mm


The height of the frontal area:

In [48]:
cnd_L3 = cnd_AR_A_fr * cnd_L1
print(f"{cnd_L3.to('mm'):~P.0f}")

275 mm


## Summary

**Evaporator**

In [49]:
print(
    f"mass flow rate of air = {evp_m_dot_air.to('kg / hr'):~P.3f}\n"
    f"mass flow rate of refrigerant = {m_dot_rfg.to('kg / hr'):~P.3f}\n"
    f"inlet air = {evp_air_in.Tdb.to('degC'):~P.2f} DB, "
    f"{evp_air_in.W.to('g / kg'):~P.3f}\n"
    f"outlet air = {evp_air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{evp_air_out.W.to('g / kg'):~P.3f}\n"
    f"inlet refrigerant = {evp_rfg_in.T.to('degC'):~P.2f}, "
    f"vapor quality {evp_rfg_in.x.to('frac'):~P.2f}\n"
    f"outlet refrigerant = {evp_rfg_out.T.to('degC'):~P.2f}\n"
    f"evaporation pressure = {evp_rfg_sat_liq.P.to('bar'):~P.2f}\n"
    f"frontal area width L1 = {evp_L1.to('mm'):~P.0f}\n"
    f"frontal area height L3 = {evp_L3.to('mm'):~P.0f}"
)

mass flow rate of air = 1500.000 kg/h
mass flow rate of refrigerant = 135.570 kg/h
inlet air = 24.00 °C DB, 9.340 g/kg
outlet air = 13.00 °C DB, 8.422 g/kg
inlet refrigerant = 10.67 °C, vapor quality 0.26 frac
outlet refrigerant = 20.67 °C
evaporation pressure = 4.24 bar
frontal area width L1 = 844 mm
frontal area height L3 = 211 mm


**Condenser**

In [50]:
print(
    f"mass flow rate of air = {cnd_m_dot_air.to('kg / hr'):~P.3f}\n"
    f"mass flow rate of refrigerant = {m_dot_rfg.to('kg / hr'):~P.3f}\n"
    f"inlet air = {T_outdoor.to('degC'):~P.2f} DB\n"
    f"outlet air = {cnd_T_air_out.to('degC'):~P.2f}\n"
    f"inlet refrigerant = {cnd_rfg_in.T.to('degC'):~P.2f}\n"
    f"outlet refrigerant = {cnd_rfg_out.T.to('degC'):~P.2f}\n"
    f"condensation pressure = {cnd_rfg_sat_liq.P.to('bar'):~P.2f}\n"
    f"frontal area width L1 = {cnd_L1.to('mm'):~P.0f}\n"
    f"frontal area height L3 = {cnd_L3.to('mm'):~P.0f}"
)

mass flow rate of air = 2454.820 kg/h
mass flow rate of refrigerant = 135.570 kg/h
inlet air = 35.00 °C DB
outlet air = 45.00 °C
inlet refrigerant = 74.32 °C
outlet refrigerant = 45.00 °C
condensation pressure = 14.92 bar
frontal area width L1 = 550 mm
frontal area height L3 = 275 mm
